In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

In [3]:
# main page with letters

next_link = "https://marvelcinematicuniverse.fandom.com/wiki/Category:Characters"

characters_link_list = []
while next_link != None:
    response = requests.get(next_link)
    html_page = response.text
    html_tree = BeautifulSoup(html_page)
    results = html_tree.find_all("a", class_="category-page__member-link")
    next_link = html_tree.find(
        "a", class_="category-page__pagination-next wds-button wds-is-secondary")
    next_link = next_link["href"] if next_link != None else None

    for result in results:
        characters_link_list.append(
            [result.text, "https://marvelcinematicuniverse.fandom.com"+result["href"]])


KeyboardInterrupt: 

In [26]:
lista_personaggi = pd.DataFrame(characters_link_list, columns=["Nome", "Link"])

In [52]:
lista_personaggi = pd.read_csv("./list_of_characters_with_biography.csv")
lista_personaggi.head()

,Nome,Link,Biografia
0,A'Lars,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Life on Titan', ""A'Lars fathere..."
1,A. Kennedy,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'First Encounter With S.H.I.E.L...."
2,A.I.M. President,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Obadiah Stane contacted the A.I..."
3,Aaron (Apple Inc.),https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Apple Inc.', 'Assisting Custome..."
4,Aaron (Baby),https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', ""Sophie walked with her son crad..."


In [16]:
lista_personaggi.head()


,Nome,Link,Biografia
0,A'Lars,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Life on Titan', ""A'Lars fathere..."
1,A. Kennedy,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'First Encounter With S.H.I.E.L...."
2,A.I.M. President,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Obadiah Stane contacted the A.I..."
3,Aaron (Apple Inc.),https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Apple Inc.', 'Assisting Custome..."
4,Aaron (Baby),https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', ""Sophie walked with her son crad..."


In [8]:
lista_personaggi.shape

(3747, 2)

In [9]:
def cerca_bio(page_tree):
    biography_list = []
    biography_initial = page_tree.find_all(id="Biography")
    description_ended = False
    if(biography_initial):
        next_span = biography_initial[0].parent.find_next()

        while not description_ended:
            if next_span.name != "a" and next_span.name != "sup":
                biography_list.append(next_span.text)

            next_span = next_span.find_next()
            if next_span.name == "h2":
                description_ended = True
            biography_list = list(dict.fromkeys(biography_list))
            lista_personaggi.at[idx, "Biografia"] = biography_list
    return biography_list

In [22]:
def cerca_serie(page_tree):
    movies_list = []
    tag_lista_film = None
    movies_initial = page_tree.find_all("div", class_="pi-item pi-data pi-item-spacing pi-border-color")
    for element in movies_initial:
        if element["data-source"] == "tv series":
            tag_lista_film = element

    if tag_lista_film != None:
        links = tag_lista_film.find_all("i")
        for link in links:
            movies_list.append([link.text, link.find_next("a")["href"]])
    return movies_list

In [18]:
def cerca_film(page_tree):
    movies_list = []
    tag_lista_film = None
    movies_initial = page_tree.find_all("div", class_="pi-item pi-data pi-item-spacing pi-border-color")
    for element in movies_initial:
        if element["data-source"] == "movie":
            tag_lista_film = element
    
    if tag_lista_film != None:
        links = tag_lista_film.find_all("i")
        for link in links:
            movies_list.append([link.text, link.find_next("a")["href"]])
    return movies_list

In [12]:
def cerca_relazione_recursive(list_item):
  if not list_item:
    return []
  next_items = list_item.find_all("li")
  if not next_items:
    return [list_item.text.split(" - ",1)]
  else:
    recursive_list = []
    for item in next_items:
      recursive_list += cerca_relazione_recursive(item)
    return recursive_list



In [34]:
def cerca_relazione(page_tree):
  relationship_initial_tag = page_tree.find(id="Relationships")
  if relationship_initial_tag:
    rel_table = relationship_initial_tag.find_next("table")
    relation_list = []
    if rel_table:
      for ul_list in rel_table.find_all("ul"):
        for list_item in ul_list.find_all("li"):
          relation_list += cerca_relazione_recursive(list_item)
    return relation_list


In [35]:
from IPython.display import clear_output

#lista_personaggi["Biografia"] = None
lista_personaggi["Film"] = None
lista_personaggi["Serie"] = None
lista_personaggi["Relazioni"] = None

num_iterazioni = 0

for idx, row in tqdm(lista_personaggi.iterrows(), total=lista_personaggi.shape[0]):
    character_page_response = requests.get(row["Link"])
    character_page_tree = BeautifulSoup(character_page_response.text)
    #biografia = cerca_bio(character_page_tree)
    film = cerca_film(character_page_tree)
    serie = cerca_serie(character_page_tree)
    relazioni = cerca_relazione(character_page_tree)
    #lista_personaggi.at[idx, "Biografia"] = biografia
    lista_personaggi.at[idx, "Film"] = film
    lista_personaggi.at[idx, "Serie"] = serie
    lista_personaggi.at[idx, "Relazioni"] = relazioni


100%|██████████| 3746/3746 [13:44<00:00,  4.54it/s]


In [42]:
lista_personaggi.to_csv("./scraping_personaggi_completo.csv", index=False)

In [65]:
lista_personaggi = pd.read_csv("./scraping_personaggi_completo.csv")
lista_personaggi.head(20)
filt = lista_personaggi.query("Nome == 'Grandmother'")
print(filt["Relazioni"])


Series([], Name: Relazioni, dtype: object)
